## 1.creates a table matching your CSV structure (address, price, property_type)

In [7]:
# Install SQL magic if not already installed
!pip install ipython-sql
%load_ext sql

# Connect to an SQLite database in memory
%sql sqlite:///:memory:


Defaulting to user installation because normal site-packages is not writeable
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
%%sql
CREATE TABLE properties (
    address VARCHAR(255),
    price VARCHAR(50),
    property_type VARCHAR(50)
);


 * sqlite:///:memory:
Done.


[]

In [3]:
import pandas as pd
from sqlalchemy import create_engine

# Read the CSV file into a DataFrame
df = pd.read_csv('database/combined_database.csv')

# Create SQLite engine and use the same in-memory database
engine = create_engine('sqlite:///:memory:')

# Load data into the SQL table
df.to_sql('properties', con=engine, if_exists='replace', index=False)


850

## Total number of properties

In [23]:
query_result = pd.read_sql('SELECT COUNT(*) AS total_properties FROM properties;', con=engine)
print(query_result)


   total_properties
0               850


## Show first 5 properties

In [6]:
import pandas as pd

# Query data using pandas
query_result = pd.read_sql('SELECT * FROM properties LIMIT 5;', con=engine)
print(query_result)


                                             Address       Price Property Type
0  Flat 102b, Larch Court 2, Royal Oak Yard, Lond...    £550,000          Flat
1  Flat 202, Block C, 27, Green Walk, London, Gre...    £810,000          Flat
2  Apartment 1111, 55, Upper Ground, London, Grea...    £800,000          Flat
3  Flat 68, Globe View House, 27, Pocock Street, ...    £165,000          Flat
4  Apartment 603, Western Building, 3, Triptych P...  £1,055,000          Flat


## Count properties by type

In [12]:
# -- Column names with spaces require special handling to avoid SQL errors.
# -- In SQL, using spaces in column names can cause confusion, as SQL interprets spaces as separators.
# -- To prevent this, we enclose column names with spaces in brackets ([ ]) for SQLite and SQL Server,
# -- or double quotes (" ") for MySQL, PostgreSQL, and Oracle.
# -- For example: [Property Type] or "Property Type"
# Query data using pandas
query_result = pd.read_sql("""
    SELECT 
        [Property Type],
        COUNT(*) as count
    FROM properties
    GROUP BY [Property Type]
    ORDER BY count DESC;
""", con=engine)

# Display the result
print(query_result)


   Property Type  count
0           Flat    637
1           flat    162
2       terraced     39
3       Terraced      6
4  semi-detached      3
5  Semi-Detached      2
6       detached      1


## Find most expensive property

In [15]:
query_result = pd.read_sql("""
SELECT *
FROM properties
ORDER BY REPLACE(REPLACE(price, '£', ''), ',', '') DESC
LIMIT 1;""", con=engine)

# Display the result
print(query_result)

                                             Address     Price Property Type
0  Flat 2508, 8, Casson Square, London, Greater L...  £998,000          Flat


## Average price by property type

In [18]:
# Execute the SQL query and format average_price as currency
query_result = pd.read_sql("""
    SELECT 
        [Property Type],
        AVG(REPLACE(REPLACE(price, '£', ''), ',', '')) as average_price
    FROM properties
    GROUP BY [Property Type]
    ORDER BY average_price DESC;
""", con=engine)

# Format the average_price as currency with commas
query_result['average_price'] = query_result['average_price'].apply(lambda x: f"£{x:,.2f}")

# Display the result
print(query_result)

   Property Type  average_price
0  semi-detached  £1,524,166.67
1       terraced  £1,367,571.79
2       Terraced  £1,263,089.00
3       detached  £1,250,000.00
4  Semi-Detached  £1,215,000.00
5           Flat    £858,582.66
6           flat    £514,378.56


## Median price of all properties

In [22]:
query_result = pd.read_sql("""
    SELECT AVG(price_numeric) AS median_price
    FROM (
        SELECT CAST(REPLACE(REPLACE(price, '£', ''), ',', '') AS FLOAT) AS price_numeric
        FROM properties
        ORDER BY price_numeric
        LIMIT 2 - (SELECT COUNT(*) FROM properties) % 2 -- Handles even/odd count
        OFFSET (SELECT (COUNT(*) - 1) / 2 FROM properties)
    );
""", con=engine)


print(query_result)


   median_price
0      670000.0


## Price distribution by ranges

In [24]:
query_result = pd.read_sql("""
SELECT 
    CASE
        WHEN CAST(REPLACE(REPLACE(price, '£', ''), ',', '') AS NUMERIC) <= 500000 THEN 'Under £500k'
        WHEN CAST(REPLACE(REPLACE(price, '£', ''), ',', '') AS NUMERIC) <= 750000 THEN '£500k-£750k' 
        WHEN CAST(REPLACE(REPLACE(price, '£', ''), ',', '') AS NUMERIC) <= 1000000 THEN '£750k-£1M'
        ELSE 'Over £1M'
    END AS price_range,
    COUNT(*) AS count
FROM properties
GROUP BY price_range
ORDER BY price_range;
""", con=engine)


print(query_result)

   price_range  count
0     Over £1M    172
1  Under £500k    242
2  £500k-£750k    263
3    £750k-£1M    173
